---
title: "Feature Engineering com séries de preços de ativos financeiros"
format:
  html:
    css: styles.css
    self-contained: true
    toc: true
    code-fold: true
    df-print: paged
editor: visual
---


------------------------------------------------------------------------

<left> ![](https://raw.githubusercontent.com/profrhozon/site/main/logo_FAE.png){width="15%"} </left>

------------------------------------------------------------------------

```         
```

::: callout-note
## Resumo

Este documento apresenta o processo de Feature Engineering aplicado a dados de séries temporais financeiras, contemplando:

-   Download de dados de commodities via `yahooquery`
-   Cálculo de log-retornos e análise de distribuições (assimetria, histograma)
-   Modelagem GARCH(1,1) para estimar variância condicional
-   Visualização de resultados utilizando `timetk` no R
:::

## 📌 Introdução: Feature Engineering em dados de séries financeiras


```{=html}
<!-- 
Escrever a intro do documento aqui ....
-->
```


### Medindo a Volatilidade: Com e Sem GARCH

A abordagem tradicional calcula a volatilidade como o desvio padrão dos retornos históricos em uma janela móvel de tamanho $N$: (dias de negociação)

$$
\sigma_{\text{hist}} = \sqrt{\frac{1}{N-1}\sum_{i=1}^{N}(r_i - \bar{r})^2}
$$

**Vantagens:**

-   Simplicidade e facilidade de implementação.

**Desvantagens:**

-   Assume volatilidade constante durante a janela. (ou seja precisa de um range dias e não é capaz de medir o desvio ou o risco/volatilidade de um dia pro outro.)

-   Não capta a persistência dos choques (efeito de clustering).

-   Não reage dinamicamente a choques recentes.

### Volatilidade com o GARCH

O modelo GARCH(1,1) estima a variância condicional de forma dinâmica:

$$
\sigma_t^2 = \omega + \alpha \epsilon_{t-1}^2 + \beta \sigma_{t-1}^2.
$$

Onde: - $\epsilon_{t-1}^2$ reflete o impacto dos choques recentes. - $\sigma_{t-1}^2$ reflete a persistência da volatilidade do período anterior. - $\omega > 0$, $\alpha \geq 0$ e $\beta \geq 0$ são parâmetros estimados.

A soma $\alpha + \beta$ mede a persistência total da volatilidade:

-   **Próximo de 1:** Choques têm efeitos duradouros; a volatilidade permanece alta por vários períodos.

-   **Menor que 1:** Os choques se dissipam mais rapidamente; a volatilidade retorna ao seu nível médio mais rápido.

**Vantagens do GARCH:**

-   Modela a volatilidade de forma dinâmica.

-   Captura o efeito de "clustering" dos choques.

-   Permite previsões mais precisas da volatilidade futura.

**Desvantagens do GARCH:**

-   Requer estimação de parâmetros e pressupõe uma estrutura específica para a volatilidade.

-   Pode ser sensível à escolha da distribuição dos resíduos (por exemplo, normal vs. $t$ de Student).

::: panel-tabset
## Python


In [ ]:
#import yfinance as yf
from yahooquery import Ticker
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
from datetime import datetime
from arch import arch_model # Lib do Python pra estimar as volatilidades (ARCH/GARCH)
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotnine import ggplot, aes, geom_line, facet_wrap, labs, theme, element_text, theme_minimal

In [ ]:
# Tickers for portfolio
TICKERS = [
  "BRFS3.SA",
  "JBSS3.SA",
  "BEEF3.SA",
  "MRFG3.SA",
  "TSN",
  "HRL",
  "GIS"
]

# Baixar os dados históricos com yahooquery
tickers = Ticker(TICKERS)
data = tickers.history(period="5y")

# Resetar o índice corretamente
data = data.reset_index()

# O yahooquery retorna um MultiIndex, então é preciso garantir que a coluna "date" exista corretamente
if "date" not in data.columns:
    raise ValueError("A coluna 'date' não foi encontrada no dataset! Verifique a estrutura do DataFrame.")

# Selecionar apenas as colunas de interesse e reformatar
portfolio_prices = data.pivot(index="date", columns="symbol", values="close").reset_index()

# Garantir que não há valores ausentes
portfolio_prices.dropna(inplace=True)

portfolio_prices.head()

Plotamos os gráficos das séries temporais de preços

```         
```


In [ ]:
#| eval: false

# Retire o comando #| eval: false pra conseguir executar essa celula dentro do Quarto
import plotly.express as px

# Certifique-se de que a coluna "date" está em formato datetime
portfolio_prices['date'] = pd.to_datetime(portfolio_prices['date'])

# Transformar os preços para o formato longo (melt) para facilitar o plot
prices_long = portfolio_prices.melt(id_vars='date', var_name='Ativo', value_name='Valor')

fig = px.line(prices_long, x='date', y='Valor', color='Ativo',
              title='Séries Temporais de Preços')
fig.show()


mas essas séries temporais tem um problema....os preços estão em seus "níveis", e isso em análise de séries temporais não é o ideal. Precisamos trabalhar com as séries estacionárias e analisar os retornos dos preços.

*A stationary time series is one whose statistical properties do not depend on the time at which the series is observed.18 Thus, time series with trends, or with seasonality, are not stationary — the trend and seasonality will affect the value of the time series at different times. On the other hand, a white noise series is stationary — it does not matter when you observe it, it should look much the same at any point in time.*

*Some cases can be confusing — a time series with cyclic behaviour (but with no trend or seasonality) is stationary. This is because the cycles are not of a fixed length, so before we observe the series we cannot be sure where the peaks and troughs of the cycles will be.*

*In general, a stationary time series will have no predictable patterns in the long-term. Time plots will show the series to be roughly horizontal (although some cyclic behaviour is possible), with constant variance.*

(Hyndman, R.J., & Athanasopoulos, G. (2021) **Forecasting: principles and practice**, 3rd edition, OTexts: Melbourne, Australia. <https://otexts.com/fpp3/stationarity.html>. Accessed on 2025/march.)

Estacionar uma série temporal econômica é importante pois:

*"Uma série temporal é dita estacionária se suas propriedades estatísticas, como a média e a variância, permanecem constantes ao longo do tempo. Em outras palavras, não há uma tendência determinística na série, e as flutuações ao redor da média são estáveis. Se a série não for estacionária, suas previsões podem ser pouco confiáveis, pois os padrões passados podem não ser representativos do futuro."* (Gujarati & Porter, 2009, p. 753).


In [ ]:
# Calcular os log-retornos
log_returns = portfolio_prices.copy()
log_returns.iloc[:, 1:] = np.log(portfolio_prices.iloc[:, 1:]).diff()

# Remover a primeira linha que contém NaN após a diferenciação
log_returns = log_returns.dropna()

```         
```


In [ ]:
#| eval: false

# Retire o comando #| eval: false pra conseguir executar essa celula dentro do Quarto
# Garantir que a coluna "date" esteja em formato datetime
log_returns['date'] = pd.to_datetime(log_returns['date'])

# Transformar para formato longo
log_returns_long = log_returns.melt(id_vars='date', var_name='Ativo', value_name='Log_Retorno')

fig = px.line(log_returns_long, x='date', y='Log_Retorno', color='Ativo',
              title='Séries Temporais de Log-Retornos')
fig.show()


Analisar as distribuições dos retornos é fundamental para entender o comportamento estatístico dos ativos financeiros. Essa análise nos permite identificar características importantes, como a presença de assimetria e caudas pesadas. <mark>Por exemplo, uma assimetria negativa indica que os retornos tendem a oscilar mais para valores baixos, sugerindo um risco maior de perdas acentuadas, enquanto uma assimetria positiva indica uma tendência para oscilações para valores mais altos</mark>. Além disso, observar a forma da distribuição ajuda a avaliar se a hipótese de normalidade é válida ou se é necessário adotar distribuições alternativas, como a distribuição $t$ de Student, que captura melhor a ocorrência de eventos extremos. Essa compreensão é crucial para a modelagem de riscos, desenvolvimento de estratégias de investimento e aprimoramento dos modelos econométricos utilizados na previsão dos preços.


In [ ]:
# Seleciona apenas as colunas dos ativos (excluindo a coluna "date")
ativos = log_returns.columns[1:]

# Calcula a assimetria para cada ativo
skewness = log_returns[ativos].skew()

# Cria um DataFrame para visualizar os resultados
skew_table = pd.DataFrame({
    'Ativo': skewness.index,
    'Skewness': skewness.values
})

# Adiciona a coluna que indica a direção da assimetria
skew_table['Direcao'] = skew_table['Skewness'].apply(
    lambda x: 'À direita' if x > 0 else ('À esquerda' if x < 0 else 'Simétrica')
)

# Exibe a tabela atualizada
skew_table

Agora teremos que ver se o range de nossa seleção de análise terá mais retornos positivos do que negativos, olhando os histogramas:

```         
```


In [ ]:
#| eval: false

# Retire o comando #| eval: false caso queira executar essa celula

import seaborn as sns
import matplotlib.pyplot as plt

# Transformar os log-retornos para formato longo (melt)
log_returns_long = log_returns.melt(id_vars=["date"], var_name="Ativo", value_name="Log_Retorno")

# Criar gráfico com Seaborn
plt.figure(figsize=(12, 8))
g = sns.FacetGrid(log_returns_long, col="Ativo", col_wrap=2, sharex=False, sharey=False)
g.map_dataframe(sns.histplot, x="Log_Retorno", kde=True, bins=30, color="black", alpha=0.5)

# Ajustar título dos gráficos
g.set_titles(col_template="{col_name}")

# Melhorar layout
plt.tight_layout()
plt.show()

Como medida de risco, utilizamos as variâncias condicionais (volatilidades) para lidar melhor com a variação diária dos log-retornos dos preços.

### Volatilidade com desvio-padrão

A volatilidade histórica pode ser medida como o desvio-padrão dos log-retornos calculado em uma janela móvel de $N$ dias de negociação.

Neste exemplo, adotamos uma janela móvel de 5 dias (aproximadamente uma semana de negociação. Para um mês de negociação, utilizar 22 e um ano 252) para capturar a volatilidade diária dos ativos.

**Vantagens:**

-   Simplicidade e facilidade de implementação.

**Desvantagens:**

-   Assume volatilidade constante durante a janela.

-   Não capta a persistência dos choques.

-   Não reage dinamicamente a choques recentes.


In [ ]:
# Calcular a volatilidade histórica com uma janela móvel de 5 dias

# Supondo que 'log_returns' já foi calculado e possui a coluna 'date' e os log-retornos dos ativos
window = 5

# Cria um DataFrame para armazenar a volatilidade histórica
vol_hist = pd.DataFrame({'date': log_returns["date"]})

# Calcula o desvio-padrão móvel (volatilidade) para cada ativo
for col in log_returns.columns[1:]:
    vol_hist[col] = log_returns[col].rolling(window=window).std()

# Exibe as ultimas linhas do DataFrame de volatilidade histórica
#print(vol_hist.head()) # 5 primeiros serão NaN
print(vol_hist.tail())

Plotando temos:

```         
```


In [ ]:
#| eval: false

# Retire o comando #| eval: false pra conseguir executar essa celula dentro do Quarto
# Certificar que "date" é datetime (já feito)
vol_hist['date'] = pd.to_datetime(vol_hist['date'])

# Transformar para formato longo
vol_hist_long = vol_hist.melt(id_vars='date', var_name='Ativo', value_name='Volatilidade_Hist')

fig = px.line(vol_hist_long, x='date', y='Volatilidade_Hist', color='Ativo',
              title='Volatilidade Histórica (Desvio-Padrão) com janela de 5 dias')
fig.show()

### Volatilidade com GARCH(1,1)

O desvio-padrão assume que precisaremos de um range maior do que 2 pontos no tempo, o que limita nossa análise pois a incompatibiliza, uma vez que precisaremos comparar os riscos diários x retornos diários (como feito anteriormente). Ou seja, não podemos comparar retornos dia-a-dia x volatilidades (risco) de 5 em 5 dias p. ex.

Os modelos heterocedásticos (da família ARCH) estimam a variância condicional dos nossos dados, ou seja, em linguagem de finanças, eles são capazes de capturar as volatilidades ou risco dos retornos dos preços de ativos financeiros ponto a ponto no tempo, ou seja, dia a dia.

O modelo GARCH(1,1) com distribuição $t$ assimétrica não está disponível diretamente na maioria das bibliotecas Python. No entanto, podemos utilizar um GARCH(1,1) com uma distribuição $t$ padrão para estimar a variância condicional. O modelo é representado por:

$$
r_t = \mu + \epsilon_t
$$

$$
\epsilon_t = \sigma_t z_t, \quad z_t \sim t_{\nu}(0, 1)
$$

$$
\sigma_t^2 = \omega + \alpha \epsilon_{t-1}^2 + \beta \sigma_{t-1}^2
$$

Onde:

-   $r_t$ é o log-retorno no tempo $t$.
-   $\mu$ é a média dos retornos.
-   $\epsilon_t$ é o termo de erro, condicionado às informações passadas.
-   $\sigma_t^2$ é a variância condicional no tempo $t$.
-   $\omega, \alpha, \beta$ são os parâmetros a serem estimados, com $\omega > 0, \alpha \geq 0, \beta \geq 0$.
-   $z_t$ segue uma distribuição $t$ de Student com $ν$ graus de liberdade para capturar as caudas pesadas observadas em retornos financeiros.

A soma $\alpha + \beta$ é frequentemente utilizada para medir a persistência da volatilidade: quanto mais próximos de 1, maior a persistência dos choques na volatilidade.

Vamos estimar a variância condicional ($\sigma^2_{t}$ ) para cada ativo:


In [ ]:
# Estimar o modelo GARCH(1,1) e salvar variância condicional
var_condicional = pd.DataFrame({"date": log_returns["date"]})

for col in log_returns.columns[1:]:
    am = arch_model(log_returns[col], vol="Garch", p=1, q=1, dist="t")
    res = am.fit(disp="off")
    var_condicional[col] = res.conditional_volatility ** 2

var_condicional.head()

Vamos avaliar os parâmetros estimados do modelo:


In [ ]:
# Inferir sobre os parâmetros do modelo GARCH(1,1) para cada ativo do portfólio

params_list = []

# Iterar sobre cada ativo (exceto a coluna 'date')
for col in log_returns.columns[1:]:
    am = arch_model(log_returns[col], vol="Garch", p=1, q=1, dist="t")
    res = am.fit(disp="off")
    
    par = res.params
    alpha_val = par.get("alpha[1]", None)
    beta_val  = par.get("beta[1]", None)
    alpha_beta_sum = (alpha_val if alpha_val is not None else 0) + (beta_val if beta_val is not None else 0)
    
    # Interpretação curta
    if alpha_beta_sum >= 0.9:
        interp = f"Alta persistência (α+β = {alpha_beta_sum:.4f})."
    else:
        interp = f"Baixa/moderada persistência (α+β = {alpha_beta_sum:.4f})."
    
    params_list.append({
         "Ativo": col,
         "mu": par.get("mu", None),
         "omega": par.get("omega", None),
         "alpha": alpha_val,
         "beta": beta_val,
         "alpha+beta": alpha_beta_sum,
         "nu": par.get("nu", None),
         "Interpretacao": interp
    })

garch_params = pd.DataFrame(params_list)

A soma $\alpha + \beta$ é um indicador crucial na modelagem GARCH para avaliar a persistência da volatilidade. Em termos práticos, os parâmetros $\alpha$ e $\beta$ têm funções distintas:

-   $\alpha$: Representa o impacto dos choques recentes (a inovação ou termo de erro $\epsilon_{t-1}^2$ na volatilidade atual. Um valor mais alto de $\alpha$ indica que choques recentes têm um efeito maior em aumentar a volatilidade.
-   $\beta$: Captura a persistência da volatilidade ao longo do tempo, ou seja, o efeito da volatilidade passada ($\sigma_{t-1}^2)$ sobre a volatilidade presente. Valores maiores de $\beta$ sugerem que a volatilidade tende a se manter elevada por um período mais longo.

Quando somamos esses dois parâmetros, ou seja, quando calculamos $\alpha + \beta$, obtemos uma medida da persistência total da volatilidade:

-   Se $\alpha + \beta$ estiver **próximo de 1**, isso indica que os choques que afetam a volatilidade têm efeitos de longa duração. Em outras palavras, um choque na volatilidade tem um impacto que se dissipa muito lentamente, mantendo a volatilidade elevada por vários períodos.
-   Se $\alpha + \beta$ for **significativamente menor que 1**, os efeitos dos choques são de curta duração e a volatilidade retorna rapidamente ao seu nível médio após um impacto.

Em alguns casos, quando $\alpha + \beta = 1$, o modelo é denominado **IGARCH** (Integrated GARCH), o que implica que os choques têm efeitos persistentes permanentemente, ou seja, a volatilidade não reverte para um valor médio fixo.

Esta característica é particularmente importante na análise de séries financeiras, pois a persistência alta da volatilidade pode implicar maior risco de mercado e desafios na previsão dos retornos futuros. Assim, a soma $\alpha + \beta$ serve como uma medida de "memória" dos choques, indicando se a volatilidade reage de forma passageira ou duradoura a eventos inesperados.

Graficamente temos:

```         
```


In [ ]:
#| eval: false

# Retire o comando #| eval: false pra conseguir executar essa celula dentro do Quarto
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Para o ativo "ZC=F"
returns_zc = log_returns[['date', 'BEEF3.SA']].copy()
vol_zc = var_condicional[['date', 'BEEF3.SA']].copy()

# Converter "date" para datetime, se necessário
returns_zc['date'] = pd.to_datetime(returns_zc['date'])
vol_zc['date'] = pd.to_datetime(vol_zc['date'])

# Criar figura com dois subplots compartilhando o eixo x
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=("Retornos Diários - BEEF3.SA", "Volatilidade Condicional (GARCH) - BEEF3.SA")
)

# Adicionar o gráfico de retornos
fig.add_trace(
    go.Scatter(x=returns_zc['date'], y=returns_zc['ZC=F'], mode='lines', name='Retornos'),
    row=1, col=1
)

# Adicionar o gráfico de volatilidade condicional
fig.add_trace(
    go.Scatter(x=vol_zc['date'], y=vol_zc['ZC=F'], mode='lines', name='Volatilidade'),
    row=2, col=1
)

fig.update_layout(
    height=600,
    width=900,
    title_text="Retorno vs. Volatilidade (GARCH) - ZC=F",
    xaxis2_title="Data",
    yaxis1_title="Retorno",
    yaxis2_title="Volatilidade Condicional"
)

fig.show()


Em alguns casos, a variância condicional pode apresentar grandes oscilações se houver outliers nos retornos ou problemas de convergência do modelo. Verifique:

-   Qualidade e limpeza dos dados
-   Resumo do ajuste (parâmetros $\alpha,\beta$ plausíveis?)
-   Distribuição ($t$ vs. normal)
-   Modelos alternativos (EGARCH, GJR-GARCH, etc.)

### Plotando retorno x risco

Aqui iremos visualizar o comportamento do ativo `ZC=F`, futuros de milho:

```         
```

Esse gráfico em Python, pode ser obtido com:


In [ ]:
#| eval: false

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Para o ativo "ZC=F"
returns_zc = log_returns[['date', 'BEEF3.SA']].copy()
vol_zc = var_condicional[['date', 'BEEF3.SA']].copy()

# Converter "date" para datetime, se necessário
returns_zc['date'] = pd.to_datetime(returns_zc['date'])
vol_zc['date'] = pd.to_datetime(vol_zc['date'])

# Criar figura com dois subplots compartilhando o eixo x
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=("Retornos Diários - BEEF3.SA", "Volatilidade Condicional (GARCH) - BEEF3.SA")
)

# Adicionar o gráfico de retornos
fig.add_trace(
    go.Scatter(x=returns_zc['date'], y=returns_zc['BEEF3.SA'], mode='lines', name='Retornos'),
    row=1, col=1
)

# Adicionar o gráfico de volatilidade condicional
fig.add_trace(
    go.Scatter(x=vol_zc['date'], y=vol_zc['BEEF3.SA'], mode='lines', name='Volatilidade'),
    row=2, col=1
)

fig.update_layout(
    height=600,
    width=900,
    title_text="Retorno vs. Volatilidade (GARCH) - BEEF3.SA",
    xaxis2_title="Data",
    yaxis1_title="Retorno",
    yaxis2_title="Volatilidade Condicional"
)

fig.show()

:::

------------------------------------------------------------------------

# References

------------------------------------------------------------------------

Gujarati, D. N., & Porter, D. C. (2009). **Basic econometrics** (5th ed.). McGraw-Hill.

Hyndman, R.J., & Athanasopoulos, G. (2021) **Forecasting: principles and practice**, 3rd edition, OTexts: Melbourne, Australia. OTexts.com/fpp3. Accessed on march 2025.\